# DASK

In [ ]:
from dask.distributed import Client, LocalCluster
from dask_jobqueue import SLURMCluster
from pathlib import Path
import os
import dask

In [ ]:
which_pc = "merlin_paper_gsa"
if 'merlin' in which_pc:
    path_dask_logs = Path('/data/user/kim_a/dask_logs')
    path_dask_logs.mkdir(parents=True, exist_ok=True)
    cluster = SLURMCluster(cores     = 8,
                           memory    ="140GB", 
                           walltime  = '23:00:00',
                           interface ='ib0',
                           local_directory = path_dask_logs.as_posix(),
                           log_directory   = path_dask_logs.as_posix(),
                           queue="daily",
                           ) 
elif 'local' in which_pc:
    cluster = LocalCluster(memory_limit='7GB') 

In [ ]:
client = Client(cluster)

In [ ]:
n_workers = 60
cluster.scale(n_workers)

In [ ]:
client

In [ ]:
client.close()
cluster.close() 

# 1. GSA setups

In [ ]:
from setups_paper_gwp import *

In [ ]:
path_base = Path('/data/user/kim_a/paper_gsa')
setup_xgbo = setup_xgbo_lca

num_params = 10000
iter_corr = 4*num_params
iter_salt = 40*num_params
iter_delt = 8*num_params
iter_xgbo = 4*num_params

n_workers_corr = 4
n_workers_salt = 39
n_workers_delt = 50
n_workers_xgbo = 4

options = {
    'corr': {
        "iterations": iter_corr,
        "n_workers":  n_workers_corr,
    }, 
    'salt': {
        "iterations": iter_salt,
        "n_workers": n_workers_salt,
    }, 
    'delt': {
        "iterations": iter_delt,
        "n_workers": n_workers_delt,
    },
    'xgbo': {
        "iterations": iter_delt,
        "n_workers": n_workers_delt,
    }
}
# gsa_corr = setup_corr(num_params, iter_corr, setup_lca_model_paper, path_base)
# gsa_salt = setup_salt(num_params, iter_salt, setup_lca_model_paper, path_base)
# gsa_delt = setup_delt(num_params, iter_delt, setup_lca_model_paper, path_base)
gsa_xgbo = setup_xgbo(num_params, iter_xgbo, setup_lca_model_paper, path_base)

In [ ]:
gsa_xgbo.perform_gsa()

# 2. Model runs

In [ ]:
# write_X_chunks(gsa_corr, n_workers_corr)
# write_X_chunks(gsa_salt, n_workers_salt)
# write_X_chunks(gsa_delt, n_workers_delt)
# write_X_chunks(gsa_xgbo, n_workers_xgbo)

In [ ]:
# # Compute model outputs for all gsa methods with dask
# task_per_worker = dask.delayed(compute_scores_per_worker)
# model_evals = []
# for option,dict_ in options.items():
#     iterations = dict_["iterations"]
#     n_workers = dict_["n_workers"]
#     for i in range(n_workers):
#         print(option, num_params, iterations, i, n_workers)
#         model_eval = task_per_worker(option, num_params, iterations, i, n_workers, setup_lca_model_paper, path_base)
#         model_evals.append(model_eval)

In [ ]:
# %%time
# dask.compute(model_evals)

## 2.5. Collect model Y chunks into one array

In [ ]:
# def generate_model_output_from_chunks(gsa, n_workers):
#     Y = np.zeros(
#         shape=(0,)
#     )
#     for i in range(n_workers):
#         filepath_Y_chunk = (
#             gsa.dirpath_Y
#             / "{}.{}.pickle".format(i, n_workers)
#         )
#         Y_chunk = read_pickle(filepath_Y_chunk)
#         Y = np.hstack(
#             [Y, Y_chunk]
#         )  # TODO change to vstack for multidimensional output
#     write_hdf5_array(Y,gsa.filepath_Y)
#     return Y

In [ ]:
# Ycorr = generate_model_output_from_chunks(gsa_corr, n_workers_corr)
# Ysalt = generate_model_output_from_chunks(gsa_salt, n_workers_salt)
# Ydelt = generate_model_output_from_chunks(gsa_delt, n_workers_delt)
# Yxgbo = generate_model_output_from_chunks(gsa_xgbo, n_workers_xgbo)

# 3. Run GSA

In [ ]:
gsa_delt.generate_unitcube_samples(return_X=False)
gsa_delt.generate_rescaled_samples(return_X=False)

In [ ]:
gsa_xgbo.generate_unitcube_samples(return_X=False)
gsa_xgbo.generate_rescaled_samples(return_X=False)

In [ ]:
%%time
gsa_corr.perform_gsa()

In [ ]:
%%time
gsa_salt.perform_gsa()

In [ ]:
%%time
gsa_delt.perform_gsa()

In [ ]:
%%time
gsa_xgbo.perform_gsa()

In [ ]:
# worker_delt = dask.delayed(gsa_delt.perform_gsa)
# model_eval_delt = worker_delt()
# worker_xgbo = dask.delayed(gsa_xgbo.perform_gsa)
# model_eval_xgbo = worker_xgbo()
# model_evals = [model_eval_delt, model_eval_xgbo]

In [ ]:
# %%time
# dask.compute(model_evals)

# 4. Validation

In [ ]:
from gsa_framework.validation import Validation
from setups_paper_gwp import *
import dask

model, write_dir, gsa_seed = setup_lca_model_paper(num_params, path_base)
validation_seed = 23467
num_influential = 60

iterations_validation = 2000
val = Validation(
    model=model,
    iterations=iterations_validation,
    seed=validation_seed,
    default_x_rescaled=None,
    write_dir=write_dir,
)

In [ ]:
# worker_validation =  dask.delayed(val.get_influential_Y_from_gsa)
worker_validation =  val.get_influential_Y_from_gsa

In [ ]:
# S_dict = gsa_corr.generate_gsa_indices()
# Scorr = abs(S_dict['spearman'])
# tag_corr = "SpearmanIndex"
# model_eval_corr = worker_validation(Scorr, num_influential, tag_corr)

# S_dict = gsa_salt.generate_gsa_indices()
# Ssalt = S_dict['Total order']
# tag_salt = "TotalIndex"
# model_eval_salt = worker_validation(Ssalt, num_influential, tag_salt)

S_dict = gsa_delt.generate_gsa_indices()
Sdelt = np.array(S_dict['delta'])
tag_delt = "DeltaIndexNr{}".format(gsa_delt.num_resamples)
model_eval_delt = worker_validation(Sdelt, num_influential, tag_delt)

# S_dict = gsa_xgbo.generate_gsa_indices()
# Sxgbo = S_dict['fscores']
# tag_xgbo = "FscoresIndex"
# model_eval_xgbo = worker_validation(Sxgbo, num_influential, tag_xgbo)

In [ ]:
model_evals = [
    model_eval_corr,
    model_eval_salt,
    model_eval_delt, 
    model_eval_xgbo,
]

In [ ]:
%%time
dask.compute(model_evals)

In [ ]:
fig_format = ['pickle']

# influential_Y_corr = val.get_influential_Y_from_gsa(Scorr, num_influential, tag=tag_corr)
# val.plot_histogram_Y_all_Y_inf(
#     influential_Y_corr, num_influential, tag=tag_corr, fig_format=fig_format
# )

# influential_Y_salt = val.get_influential_Y_from_gsa(Ssalt, num_influential, tag=tag_salt)
# val.plot_histogram_Y_all_Y_inf(
#     influential_Y_salt, num_influential, tag=tag_salt, fig_format=fig_format
# )

influential_Y_delt = val.get_influential_Y_from_gsa(Sdelt, num_influential, tag=tag_delt)
val.plot_histogram_Y_all_Y_inf(
    influential_Y_delt, num_influential, tag=tag_delt, fig_format=fig_format
)

# influential_Y_xgbo = val.get_influential_Y_from_gsa(Sxgbo, num_influential, tag=tag_xgbo)
# val.plot_histogram_Y_all_Y_inf(
#     influential_Y_xgbo, num_influential, tag=tag_xgbo, fig_format=fig_format
# )

# XGBoost tuning

In [ ]:
%%time

from gsa_framework.test_functions import Morris4
from gsa_framework.methods.gradient_boosting import GradientBoosting
from pathlib import Path
from sklearn.model_selection import GridSearchCV, train_test_split
import numpy as np
import xgboost as xgb
from sklearn.metrics import explained_variance_score, r2_score
from setups_paper_gwp import setup_lca_model_paper
from gsa_framework.utils import read_hdf5_array

if __name__ == "__main__":

    path_base = Path('/data/user/kim_a/paper_gsa/')
#     path_base = Path("/Users/akim/PycharmProjects/gsa_framework/dev/write_files/")

    # 1. Models
    num_params = 10000
    model, write_dir, gsa_seed = setup_lca_model_paper(path_base, num_params)
    filepath_Xr = write_dir / "arrays" / "X.rescaled.randomSampling.40000.92374523.hdf5"
    filepath_Y = write_dir / "arrays" / "Y.randomSampling.40000.92374523.hdf5"
    X = read_hdf5_array(filepath_Xr)
    Y = read_hdf5_array(filepath_Y).flatten()
    fig_format = []  # can have elements "pdf", "html", "pickle"

    test_size = 0.2

    option = "gsa"
    if "tuning" in option:
        # 1. Preparations
        np.random.seed(gsa_seed)
#         X_unitcube = np.random.rand(iterations, num_params)
#         X = model.rescale(X_unitcube)
#         Y = model(X)
        # 2. Prepare training and testing sets for  gradient boosting trees
        X_train, X_test, Y_train, Y_test = train_test_split(
            X,
            Y,
            test_size=test_size,
            random_state=gsa_seed,
        )

        dtrain = xgb.DMatrix(X_train, Y_train)
        X_dtest = xgb.DMatrix(X_test)

        if option == "tuning":
            ### ROUND 1 ###
            # xgb.train uses parameter `num_boost_round`, while XGBRegressor needs `n_estimators`. These two are the same.
            param_grid = {
                "learning_rate": [0.15],
                "gamma": [0],
                "min_child_weight": [60, 100, 140],
                "max_depth": [2],
                "reg_lambda": [0, 10],
                "reg_alpha": [0, 10],
                "n_estimators": [500, 800, 1100],
                "subsample": [0.3, 0.6],
                "colsample_bytree": [0.3, 0.6],
            }

            optimal_params = GridSearchCV(
                estimator=xgb.XGBRegressor(
                    objective="reg:squarederror",
                    seed=gsa_seed,
                ),
                param_grid=param_grid,
                scoring="explained_variance",  # explained_variance takes into account mean squared error, r2 does not. former is unbiasede, so better than r2
                cv=3,
            )
            optimal_params.fit(
                X_train,
                Y_train,
                early_stopping_rounds=10,
                eval_set=[(X_test, Y_test)],
                verbose=False,
            )

            print(optimal_params.best_params_)

            import pickle

            filepath = write_dir / "arrays" / "optimal_params_round_1.pickle"
            if filepath.exists():
                filepath = write_dir / "arrays" / "optimal_params_round_2.pickle"
            with open(filepath, "wb") as f:
                pickle.dump(optimal_params, f)

        elif option == "no tuning":
            np.random.seed(None)
            reg = xgb.XGBRegressor(
                verbosity=1,  # 0 (silent), 1 (warning), 2 (info), 3 (debug)
                objective="reg:squarederror",
                seed=gsa_seed,
                learning_rate=0.15,
                gamma=0,
                min_child_weight=300,
                max_depth=4,
                reg_lambda=0,
                reg_alpha=0,
                n_estimators=600,
                subsample=0.3,
                colsample_bytree=0.3,
            )
            reg.fit(X_train, Y_train)
            ev_train = explained_variance_score(reg.predict(X_train), Y_train)
            ev_test = explained_variance_score(reg.predict(X_test), Y_test)
            print(ev_train, ev_test)

    else:
        iterations = 4 * num_params
        tuning_parameters = dict(
            learning_rate=0.15,
            gamma=0,
            min_child_weight=300,
            max_depth=4,
            reg_lambda=0,
            reg_alpha=0,
            n_estimators=600,
            subsample=0.3,
            colsample_bytree=0.3,
        )
        gsa = GradientBoosting(
            iterations=iterations,
            model=model,
            write_dir=write_dir,
            seed=gsa_seed,
            tuning_parameters=tuning_parameters,
            test_size=test_size,
            xgb_model=None,
        )

        S_dict = gsa.perform_gsa(flag_save_S_dict=True)
        print(S_dict["stat.r2"], S_dict["stat.explained_variance"])
        gsa.plot_sa_results(
            {"fscores": S_dict["fscores"]},
            fig_format=fig_format,
        )


# Stability

## 1. Correlation coefficients

In [ ]:
from setups_paper_gwp import *
from copy import deepcopy
from gsa_framework.sensitivity_analysis.correlations import corrcoef_parallel_stability_spearman

In [ ]:
# read X and Y
num_params = 10000
iter_corr = 4*num_params
gsa_corr = setup_corr(num_params, iter_corr, setup_lca_model_paper, path_base)
X_rescaled = read_hdf5_array(gsa_corr.filepath_X_rescaled)
Y = read_hdf5_array(gsa_corr.filepath_Y).flatten()

gsa = deepcopy(gsa_corr)
num_steps = 50
num_bootstrap = 60

# Convergence class
conv = Convergence(
    gsa.filepath_Y,
    gsa.num_params,
    gsa.generate_gsa_indices,
    gsa.gsa_label,
    gsa.write_dir,
    num_steps=num_steps,
)

write_dir_stability = gsa.write_dir / 'stability_intermediate_{}'.format(gsa.gsa_label)
write_dir_stability.mkdir(parents=True, exist_ok=True)
# Generate random seeds
np.random.seed(gsa.seed)
stability_seeds = np.random.randint(
    low=0,
    high=2147483647,
    size=(len(conv.iterations_for_convergence), num_bootstrap),
)

In [ ]:
%%time
filename_S = "stability.S.{}.{}.{}Step{}.{}.{}.pickle".format(
    gsa.gsa_label, gsa.sampling_label, gsa.iterations, conv.iterations_step, num_bootstrap, gsa.seed,
)
filepath_S = gsa.write_dir / "arrays" / filename_S
if filepath_S.exists():
    print("--> {} already exists".format(filename_S))
    S_dict_stability = read_pickle(filepath_S)
else:
    S_dict_stability = {}
    for i,iterations_current in enumerate(conv.iterations_for_convergence):
        S_array = np.zeros([0,num_params])
        print("{}".format(iterations_current))
        filename_S_current = "S.{}Step{}.{}.{}.pickle".format(iterations_current,conv.iterations_step,num_bootstrap,gsa.seed)
        filepath_S_current = write_dir_stability / filename_S_current
        if filepath_S_current.exists():
            print("--> {} already exists".format(filename_S_current))
            S_dict = read_pickle(filepath_S_current)
        else:
            for j in range(num_bootstrap):
                stability_seed = stability_seeds[i,j]
                np.random.seed(stability_seed)
                choice = np.random.choice(np.arange(gsa.iterations), iterations_current, replace=False)
                Y_current = Y[choice]
                X_current = X_rescaled[choice,:]
                S_current = corrcoef_parallel_stability_spearman(Y_current, X_current)['spearman']
                S_array = np.vstack([S_array, S_current])
            S_dict = {iterations_current: {"spearman": S_array}}
            write_pickle(S_dict, filepath_S_current)
        S_dict_stability.update(S_dict)
    write_pickle(S_dict_stability, filepath_S)



## 2. Delta and xgboost

In [ ]:
from setups_paper_gwp import *
from copy import deepcopy
from gsa_framework.lca import LCAModel
from gsa_framework.sensitivity_analysis.delta_moment import delta_moment_stability
from gsa_framework.sensitivity_analysis.gradient_boosting import xgboost_scores_stability
from gsa_framework.methods.delta_moment import DeltaMoment
from gsa_framework.methods.gradient_boosting import GradientBoosting
from gsa_framework.convergence import Convergence
from gsa_framework.utils import *
from gsa_framework.sampling.get_samples import latin_hypercube_samples
from pathlib import Path
import time
import warnings
warnings.filterwarnings("ignore")

path_base = Path('/data/user/kim_a/paper_gsa/')
setup_xgbo = setup_xgbo_lca

In [ ]:
def compute_per_worker_delt(num_params, iterations_current, stability_seed):
    # This part might be different for morris and lca
    iter_delt = 8*num_params
    gsa_delt = setup_delt(num_params, iter_delt, setup_lca_model_paper, path_base)
    # The rest is the same for both
    filepath_Y = gsa_delt.write_dir_stability / "Y.step{}.seed{}.pickle".format(iterations_current, stability_seed)
    Y = read_pickle(filepath_Y).flatten()
    X = latin_hypercube_samples(gsa_delt.iterations, gsa_delt.num_params, seed=gsa_delt.seed)
    np.random.seed(stability_seed)
    choice = np.random.choice(np.arange(gsa_delt.iterations), iterations_current, replace=False)
    Xr = gsa_delt.model.rescale(X[choice, :])
    del X
    filepath_S = gsa_delt.write_dir_stability / "S.step{}.seed{}.pickle".format(iterations_current, stability_seed)
    if not filepath_S.exists():
        S_dict = delta_moment_stability(
            Y, Xr, num_resamples=gsa_delt.num_resamples, seed=stability_seed
        )
        write_pickle(S_dict, filepath_S)
    else:
        print("{} already exists".format(filepath_S.name))
        S_dict = read_pickle(filepath_S)
    
    return S_dict

def compute_per_worker_xgbo(num_params, iterations_current, stability_seed):
    iter_xgbo =4*num_params
    gsa_xgbo = setup_xgbo(num_params, iter_xgbo, setup_lca_model_paper, path_base)
    filepath_Y = gsa_xgbo.write_dir_stability / "Y.step{}.seed{}.pickle".format(iterations_current, stability_seed)
    Y = read_pickle(filepath_Y).flatten()
    np.random.seed(gsa_xgbo.seed)
    X = np.random.rand(iter_xgbo, num_params)
    np.random.seed(stability_seed)
    choice = np.random.choice(np.arange(gsa_xgbo.iterations), iterations_current, replace=True)
    Xr = gsa_xgbo.model.rescale(X[choice, :])
    del X
    filepath_S = gsa_xgbo.write_dir_stability / "S.step{}.seed{}.pickle".format(iterations_current, stability_seed)
    if not filepath_S.exists():
        S_dict = xgboost_scores_stability(
            Y, 
            Xr, 
            tuning_parameters=gsa_xgbo.tuning_parameters,
            test_size=gsa_xgbo.test_size,
            xgb_model = gsa_xgbo.xgb_model,
        )
        write_pickle(S_dict, filepath_S)
    else:
        print("{} already exists".format(filepath_S.name))
        S_dict = read_pickle(filepath_S)
    
    return S_dict

In [ ]:
num_params = 10000
iter_delt = 8*num_params
gsa_delt = setup_delt(num_params, iter_delt, setup_lca_model_paper, path_base)
iter_xgbo = 4*num_params
gsa_xgbo = setup_xgbo(num_params, iter_xgbo, setup_lca_model_paper, path_base)

num_steps = 50
num_bootstrap = 60

option = 'xgboost'
if option=='delta':
    gsa = gsa_delt
    compute_per_worker = compute_per_worker_delt
elif option=='xgboost':
    gsa = gsa_xgbo
    compute_per_worker = compute_per_worker_xgbo

task_per_worker = dask.delayed(compute_per_worker)
# task_per_worker = compute_per_worker

In [ ]:
# Test
# compute_per_worker_xgbo(num_params, 9600, 901875159)

In [ ]:
# n_workers = 60

conv = Convergence(
    gsa.filepath_Y,
    gsa.num_params,
    gsa.generate_gsa_indices,
    gsa.gsa_label,
    gsa.write_dir_convergence,
    num_steps=num_steps,
)

np.random.seed(gsa.seed)
stability_seeds = np.random.randint(
    low=0,
    high=2147483647,
    size=(len(conv.iterations_for_convergence), num_bootstrap),
)

Y = read_hdf5_array(gsa.filepath_Y).flatten()

num_times = n_workers // num_bootstrap
model_evals = []
i = 0
for i_iter in range(len(conv.iterations_for_convergence)//num_times+1):
    iterations_current_multiple = conv.iterations_for_convergence[i_iter*num_times:(i_iter+1)*num_times]
    model_evals_bootstrap_j_k = []
    for iterations_current in iterations_current_multiple:
        model_evals_bootstrap_j = []
        for j in range(num_bootstrap):
            stability_seed = stability_seeds[i,j]
            np.random.seed(stability_seed)
            choice = np.random.choice(np.arange(gsa.iterations), iterations_current, replace=True) 
            # Write Y
            filepath_Y_ij = gsa.write_dir_stability / "Y.step{}.seed{}.pickle".format(iterations_current, stability_seed)
            if not filepath_Y_ij.exists():
                Y_ij = Y[choice]
                write_pickle(Y_ij, filepath_Y_ij)
            else:
    #             print("{} already exists".format(filepath_Y_ij.name))  
                pass
            # Model evals
            filepath_S_current = gsa.write_dir_stability / "S.step{}.seed{}.pickle".format(iterations_current, stability_seed)
            if not filepath_S_current.exists():
                model_eval = task_per_worker(num_params, iterations_current, stability_seed)
                model_evals_bootstrap_j.append(model_eval)
        model_evals_bootstrap_j_k += model_evals_bootstrap_j
        i += 1
    if len(model_evals_bootstrap_j_k) > 0:
        model_evals.append(model_evals_bootstrap_j_k)

In [ ]:
%%time
for i,model_evals_bootstrap_j_k in enumerate(model_evals):
    print(i)
    dask.compute(model_evals_bootstrap_j_k)

In [ ]:
# Collect all results
def create_stability_dict_delt(num_params, iterations_for_convergence, stability_seeds):
    iter_delt = 8*num_params
    gsa_delt = setup_delt(num_params, iter_delt, setup_lca_model_paper)
    iterations_step = iterations_for_convergence[1] - iterations_for_convergence[0]
    num_bootstrap = stability_seeds.shape[1]
    filename_S_stability = "stability.S.{}.{}.{}Step{}.{}.{}.pickle".format(
    gsa_delt.gsa_label, gsa_delt.sampling_label, gsa_delt.iterations, iterations_step, num_bootstrap, gsa_delt.seed,
    )
    filepath_S_stability = gsa_delt.write_dir / 'arrays' / filename_S_stability
    if filepath_S_stability.exists():
        print("{} already exists".format(filepath_S_stability.name))  
        S_dict = read_pickle(filepath_S_stability)
    else:
        S_dict = {}
        for i,iterations_current in enumerate(iterations_for_convergence):
            S_array = np.zeros((0,num_params))
            for j in range(num_bootstrap):
                stability_seed = stability_seeds[i,j]
                filepath_S = \
                gsa_delt.write_dir_stability / "S.step{}.seed{}.pickle".format(iterations_current, stability_seed)
                if not filepath_S.exists():
                    print("{} does not exist".format(filepath_S.name))
                    return
                else:
                    S_current = read_pickle(filepath_S)
                    S_array = np.vstack([S_array, S_current['delta']])
            S_dict[iterations_current] = {"delta": S_array}
        write_pickle(S_dict, filepath_S_stability)
    return S_dict

def create_stability_dict_xgbo(num_params, iterations_for_convergence, stability_seeds):
    iter_xgbo = 4*num_params
    gsa_xgbo = setup_xgbo(num_params, iter_xgbo, setup_lca_model_paper, path_base)
    iterations_step = iterations_for_convergence[1] - iterations_for_convergence[0]
    num_bootstrap = stability_seeds.shape[1]
    filename_S_stability = "stability.S.{}.{}.{}Step{}.{}.{}.pickle".format(
    gsa_xgbo.gsa_label, gsa_xgbo.sampling_label, gsa_xgbo.iterations, iterations_step, num_bootstrap, gsa_xgbo.seed,
    )
    filepath_S_stability = gsa_xgbo.write_dir / 'arrays' / filename_S_stability
    if filepath_S_stability.exists():
        print("{} already exists".format(filepath_S_stability.name))  
        S_dict = read_pickle(filepath_S_stability)
    else:
        S_dict = {}
        for i,iterations_current in enumerate(iterations_for_convergence):
            S_array = np.zeros((0,num_params))
            for j in range(num_bootstrap):
                stability_seed = stability_seeds[i,j]
                filepath_S = \
                gsa_xgbo.write_dir_stability / "S.step{}.seed{}.pickle".format(iterations_current, stability_seed)
                if not filepath_S.exists():
                    print("{} does not exist".format(filepath_S.name))
                    return
                else:
                    S_current = read_pickle(filepath_S)
                    S_array = np.vstack([S_array, S_current['fscores']])
            S_dict[iterations_current] = {"fscores": S_array}
        write_pickle(S_dict, filepath_S_stability)
    return S_dict

In [ ]:
%%time
S_dict = create_stability_dict_xgbo(num_params, conv.iterations_for_convergence, stability_seeds)

## 3. Sobol

In [ ]:
from gsa_framework.convergence import Convergence
from pathlib import Path
from gsa_framework.utils import *
from setups_paper_gwp import setup_salt, setup_lca_model_paper
from gsa_framework.sensitivity_analysis.saltelli_sobol import *

In [ ]:
# 1. Choose which stability dictionaries to include
path_base = Path('/data/user/kim_a/paper_gsa')

if __name__ == "__main__":

    # Sobol stability dictionaries
    num_params = 10000
    iterations = 40 * num_params
    num_steps = 50
    num_bootstrap = 60

    gsa = setup_salt(num_params, iterations, setup_lca_model_paper, path_base)

    # Convergence class
    conv = Convergence(
        gsa.filepath_Y,
        gsa.num_params,
        gsa.generate_gsa_indices,
        gsa.gsa_label,
        gsa.write_dir,
        num_steps=num_steps,
    )
    stability_seeds = np.random.randint(
        low=0,
        high=2147483647,
        size=(len(conv.iterations_for_convergence), num_bootstrap),
    )

    filename_S = "stability.S.{}.{}.{}Step{}.{}.{}.pickle".format(
        gsa.gsa_label,
        gsa.sampling_label,
        gsa.iterations,
        conv.iterations_step,
        num_bootstrap,
        gsa.seed,
    )
    filepath_S = gsa.write_dir / "arrays" / filename_S
    if filepath_S.exists():
        print("--> {} already exists".format(filename_S))
        S_dict_stability = read_pickle(filepath_S)
    else:
        Y = read_hdf5_array(gsa.filepath_Y).flatten()
        S_dict_stability = sobol_indices_stability(
            Y,
            num_params,
            conv.iterations_for_convergence,
            num_bootstrap,
            stability_seeds,
        )
        write_pickle(S_dict_stability, filepath_S)

## 4. XGBoost

# Standardized regression coefficients


In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
from setups_paper_gwp import setup_corr, setup_delt, setup_lca_model_paper
from pathlib import Path
from gsa_framework.utils import read_hdf5_array

Using environment variable BRIGHTWAY2_DIR for data directory:
/data/user/kim_a/Brightway3


In [2]:
path_base = Path('/data/user/kim_a/paper_gsa')

num_params = 10000
iter_corr = 4*num_params
gsa_corr = setup_corr(num_params, iter_corr, setup_lca_model_paper, path_base)
S_corr = gsa_corr.perform_gsa()

iter_delt = 8*num_params
gsa_delt = setup_delt(num_params, iter_delt, setup_lca_model_paper, path_base)
S_delt = gsa_delt.perform_gsa()

Unitcube samples ->    0.000 s
Rescaled samples ->    0.000 s
Model outputs    ->    0.000 s
GSA indices      ->    0.012 s
Total GSA time   ->    0.013 s 

Unitcube samples ->    0.000 s
Rescaled samples ->    0.001 s
Model outputs    ->    0.000 s
GSA indices      ->    0.026 s
Total GSA time   ->    0.027 s 



In [3]:
%%time
X = read_hdf5_array(gsa_corr.filepath_X_rescaled)
Y = read_hdf5_array(gsa_corr.filepath_Y).flatten()
reg_4x = LinearRegression().fit(X, Y)
reg_4x.score(X, Y)

CPU times: user 42min 46s, sys: 7min 34s, total: 50min 20s
Wall time: 2min 48s


0.9762511918956349

In [6]:
src_4x = reg_4x.coef_ * np.std(X, axis=0) / np.std(Y)
np.sum(src_4x**2)

0.994671581728666

In [23]:
fig = gsa_corr.plot_sa_results({"reg_4x": src_4x})

In [8]:
S_corr.pop("pearson")
fig = gsa_corr.plot_sa_results(S_corr)

In [9]:
%%time
X = read_hdf5_array(gsa_delt.filepath_X_rescaled)
Y = read_hdf5_array(gsa_delt.filepath_Y).flatten()
reg_8x = LinearRegression().fit(X, Y)
reg_8x.score(X, Y)

CPU times: user 1h 6s, sys: 9min 35s, total: 1h 9min 41s
Wall time: 4min 3s


0.9802049934304004

In [11]:
src_8x = reg_8x.coef_ * np.std(X, axis=0) / np.std(Y)
np.sum(src_8x**2)

0.9811611169918748

In [21]:
fig = gsa_delt.plot_sa_results({"reg_8x": src_8x**2})

In [13]:
S_delt.pop("delta_conf")
fig = gsa_delt.plot_sa_results(S_delt)

In [17]:
%%time
X1 = X[:num_params, :]
Y1 = Y[:num_params]
reg_1x = LinearRegression().fit(X1, Y1)
reg_1x.score(X1, Y1)

CPU times: user 345 ms, sys: 109 ms, total: 454 ms
Wall time: 136 ms


0.9940257127566274

In [18]:
src_1x = reg_1x.coef_ * np.std(X1, axis=0) / np.std(Y1)
np.sum(src_1x**2)

1.1484823037926393

In [19]:
%%time
X2 = X[:2*num_params, :]
Y2 = Y[:2*num_params]
reg_2x = LinearRegression().fit(X2, Y2)
reg_2x.score(X2, Y2)

CPU times: user 34min 24s, sys: 4min 44s, total: 39min 8s
Wall time: 2min 3s


0.9883848660201295

In [20]:
src_2x = reg_2x.coef_ * np.std(X2, axis=0) / np.std(Y2)
np.sum(src_2x**2)

0.9880823868917136

# Archived

# 1. Construct LCA model

In [ ]:
from gsa_framework.lca import LCAModel
from gsa_framework.methods.correlations import CorrelationCoefficients
from gsa_framework.methods.extended_FAST import eFAST
from gsa_framework.methods.saltelli_sobol import SaltelliSobol
from gsa_framework.methods.gradient_boosting import GradientBoosting
from gsa_framework.validation import Validation
from pathlib import Path
import brightway2 as bw
import time
import numpy as np
from gsa_framework.plotting import histogram_Y1_Y2
from gsa_framework.utils import read_hdf5_array

if __name__ == "__main__":

#     path_base = Path(
#         "/Users/akim/PycharmProjects/gsa_framework/dev/write_files/paper_gsa/"
#     )
    path_base = Path('/data/user/kim_a/paper_gsa/gsa_framework_files')

    # LCA model
    bw.projects.set_current("GSA for paper")
    co = bw.Database("CH consumption 1.0")
    demand_act = [act for act in co if "Food and non-alcoholic beverages sector" in act['name']][0]
    print(demand_act)
    demand = {demand_act: 1}
    method = ("IPCC 2013", "climate change", "GWP 100a")

    # Define some variables
    num_params = 162299
    iterations_validation = 2000
    write_dir = path_base / "lca_model_food_{}".format(num_params)
    model = LCAModel(demand, method, write_dir) # TODO add num_params later
    gsa_seed = 3403
    validation_seed = 7043
    fig_format = ["html", "pickle"]

    # Make sure  that the chosen num_params in LCA are appropriate
    val = Validation(
        model=model,
        iterations=iterations_validation,
        seed=4444,
        default_x_rescaled=model.default_uncertain_amounts,
        write_dir=write_dir,
    )
    num_params_paper = 10000
    tag = "numParams{}".format(num_params_paper)
    scores_dict = model.get_lsa_scores_pickle(model.write_dir / "LSA_scores")
    uncertain_tech_params_where_subset, _ = model.get_nonzero_params_from_num_params(scores_dict, num_params_paper)
    parameter_choice = []
    for u in uncertain_tech_params_where_subset:
        where_temp = np.where(model.uncertain_tech_params_where == u)[0]
        assert len(where_temp) == 1
        parameter_choice.append(where_temp[0])
    parameter_choice.sort()

In [ ]:
Y_subset = val.get_influential_Y_from_parameter_choice(parameter_choice=parameter_choice, tag=tag)
val.plot_histogram_Y_all_Y_inf(Y_subset, num_influential=num_params_paper)